In [ ]:
# Import packages
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
filepath = "Data/{filename}.csv" # TO UPDATE after feature engineering is complete

# Column names from feature engineered dataset
response_col_name = "" # Response col
cat_col_names = [] # Categorical col 
num_col_names = [] # Numerical col

In [ ]:
def preprocess_dataframe(
        df : pd.DataFrame, 
        response_col_name : str,
        cat_col_names : list[str] = [], 
        num_col_names : list[str] = [],
        scale : bool = True,
        OHE : bool = True
        ) -> pd.DataFrame:
    """
    For scaling and one-hot encoding dataframe.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        response_col_name (str): Name of response column
        cat_col_names (list[str]): List of categorical column names to evaluate.
        num_col_names (list[str]): List of numeric column names to evaluate.
        scale (bool): Indicate if data should be scaled
        OHE (bool): Indicate if categorical variables should undergo One-Hot Encoding

    Returns:
        pd.DataFrame: A DataFrame with columns scaled/One-Hot Encoded
    """
    
    y = df[response_col_name].values # Response column
    X_num = df[num_col_names].values # Numerical columns
    X_cat = df[cat_col_names].values # Categorical columns

    if scale:
        # Scale numerical features
        scaler = StandardScaler()
        X_num = scaler.fit_transform(X_num)

    if OHE:
        # One-hot encode categorical features
        encoder = OneHotEncoder(sparse = False, drop = 'first')  # drop = 'first' to avoid dummy variable trap
        X_cat = encoder.fit_transform(X_cat)
    
    # Combine the scaled numerical and encoded categorical features
    X_preprocessed = pd.DataFrame(
        data = np.hstack((X_num, X_cat)),  # Horizontal stack to combine arrays
        columns = num_col_names + list(encoder.get_feature_names_out(cat_col_names))
    )

    # Create a new DataFrame including the response variable
    preprocessed_df = pd.DataFrame(X_preprocessed, columns = X_preprocessed.columns)
    preprocessed_df[response_col_name] = y

    return preprocessed_df
    

In [ ]:
def principal_component_analysis(
        df : pd.DataFrame, 
        response_col_name : str,
        cat_col_names : list[str] = [], 
        num_col_names : list[str] = [],
        var : float = 0.95, 
        logs : bool = False
        ) -> pd.DataFrame:
    """
    For reducing the dimensions of a dataframe using PCA.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        response_col_name (str): Name of response column
        cat_col_names (list[str]): List of categorical column names to evaluate.
        num_col_names (list[str]): List of numeric column names to evaluate.
        var (float): Proportion of variance that should be preserved
        logs (bool): Indicate if logs should be printed

    Returns:
        pd.DataFrame: A DataFrame that has undergone PCA
    """

    y = df[response_col_name].values # Response column
    X_num = df[num_col_names].values # Numerical columns
    X_cat = df[cat_col_names].values # Categorical columns

    pca = PCA(n_components = var) # Keep 'var' proportion of the variance : default 95%
    X_pca = pca.fit_transform(X_num) 

    pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
    df_pca = pd.DataFrame(X_pca, columns = pca_columns)

    df_modified = pd.concat(
        [
            df_pca, 
            pd.DataFrame(X_cat, columns = cat_col_names), 
            pd.Series(y, name = response_col_name)
        ], 
        axis = 1)
    
    if logs:
        print(f"Number of components selected: {pca.n_components_}")
        print("Explained variance ratio for each component:", pca.explained_variance_ratio_)
        print("Cumulative explained variance:", pca.explained_variance_ratio_.cumsum())
        print("Final DataFrame with PCA applied to numeric columns:")
        print(df_modified.head())

    return df_modified

In [ ]:
def filter_low_variance(
        df: pd.DataFrame, 
        num_col_names: list[str], 
        threshold: float = 0.1
        ) -> pd.DataFrame:
    """
    Filter numeric columns based on a variance threshold.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        num_col_names (list[str]): List of numeric column names to evaluate.
        threshold (float): The variance threshold for filtering columns.

    Returns:
        pd.DataFrame: A DataFrame with low variance numeric columns removed.
    """
    
    # Calculate the variance for each numeric column
    variances = df[num_col_names].var()

    # Filter columns with variance greater than the threshold : default 0.1
    high_variance_cols = variances[variances > threshold].index.tolist()

    # Create a new DataFrame with only the selected columns
    filtered_df = df[high_variance_cols]

    return filtered_df

In [ ]:
def undersample_data(
        df: pd.DataFrame, 
        response_col_name : str
        ) -> pd.DataFrame:
    """
    Perform undersampling to balance the classes in the dataset.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing the data.
        target_col (str): The name of the target column (class label).

    Returns:
        pd.DataFrame: A balanced DataFrame with equal samples from both classes.
    """
    # Separate the majority and minority classes
    majority_class = df[response_col_name].value_counts().idxmax()
    minority_class = df[response_col_name].value_counts().idxmin()

    # Split the DataFrame into majority and minority
    df_majority = df[df[response_col_name] == majority_class]
    df_minority = df[df[response_col_name] == minority_class]

    # Randomly sample from the majority class
    df_majority_sampled = df_majority.sample(n = len(df_minority), random_state = 42)

    # Combine the minority class with the undersampled majority class
    df_balanced = pd.concat([df_majority_sampled, df_minority])

    # Shuffle the resulting DataFrame
    df_balanced = df_balanced.sample(frac = 1, random_state = 42).reset_index(drop = True)

    return df_balanced